**Install Packages**

In [ ]:
!pip install pycaret
!pip install mysql-connector-python
!pip install sqlalchemy==1.4.32

**Import Packages**

In [ ]:
import pycaret
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector
import sys

from pycaret import classification
from datetime import datetime
from google.colab.data_table import DataTable
from google.colab import files
from google.colab import drive

sys.path.append("/content/drive/MyDrive")

drive.mount('/content/drive')
DataTable.max_columns = 100

def odds_calculator(probability):

    return round(-100 / ((1/probability)-1))

def juiced_odds_calculator(probability):

    return round(-100 / ((1/probability)-1)) - 15

Mounted at /content/drive


**Import Data**

In [ ]:
engine = sqlalchemy.create_engine()
Player_Positions = pd.read_csv("/content/drive/MyDrive/Lineup_MLB_Player_Positions.csv")

#Exclude_Variables = ["Unnamed: 0","pitching_numberOfPitches","pitching_outs","pitching_strikes","pitching_strikePercentage","pitching_totalBases","pitching_baseOnBalls","pitching_hitByPitch","pitching_atBats",       'batting_groundOuts','batting_hitByPitch', 'batting_caughtStealing','batting_stolenBases','batting_plateAppearances',"pitching_game_id","batting_game_id",'batting_leftOnBase', 'batting_sacBunts', 'batting_sacFlies',"batting_numberOfPitches","batting_totalBases","batting_baseOnBalls", "batting_doubles", "batting_triples","batting_homeRuns","batting_airOuts","batting_atBats", "batting_rbi"]
#file_string = f"prediction-2023-05-09.csv"
file_string = f"prediction-{datetime.today().strftime('%Y-%m-%d')}.csv"
Real_Features = ["pitching_gamesPlayed","pitching_runs","pitching_era", "pitching_strikeOuts", "pitching_hits","pitching_id","pitching_name", "batting_gamesPlayed", "batting_runs","batting_strikeOuts", "batting_hits","batting_rbi","batting_id","batting_name","temp", "game_id", "game_venue","game_date"]       

Daily_Lineup_Data = pd.read_csv(f"/content/drive/MyDrive/{file_string}")
Daily_Lineup_Data = Daily_Lineup_Data[Real_Features]
print(file_string, Daily_Lineup_Data.info())

**Load Model**

In [ ]:
hit_model_file_string = f"{datetime.today().strftime('%Y-%m-%d')} Batter Hit LR Model"
#hit_model_file_string = f"2023-05-09 Batter Hit LR Model"
Hit_Model = pycaret.classification.load_model(f"/content/drive/MyDrive/{hit_model_file_string}")

print(hit_model_file_string)

Transformation Pipeline and Model Successfully Loaded
2023-05-13 Batter Hit LR Model


**Get Most Confident Predictions**

In [9]:
Daily_Hit_Prediction = pycaret.classification.predict_model(data = Daily_Lineup_Data, estimator = Hit_Model)
Daily_Hit_Prediction = pd.merge(Daily_Hit_Prediction, Player_Positions[['player_id','player_team_name']], left_on='batting_id', right_on='player_id', how='left')

Daily_Hit_Prediction["theo_odds"] = Daily_Hit_Prediction["prediction_score"].apply(odds_calculator)
Daily_Hit_Prediction["theo_juiced_odds"] = Daily_Hit_Prediction["prediction_score"].apply(juiced_odds_calculator)

Confident_Hit_Predicitions = Daily_Hit_Prediction[(Daily_Hit_Prediction["prediction_score"] >= 0.65) & (Daily_Hit_Prediction["prediction_label"] == 1)].sort_values(by = "prediction_score", ascending = False).drop_duplicates("batting_name")
Confident_Hit_Predicitions[["batting_name",	"prediction_label",	"prediction_score", "player_team_name", "theo_odds", "theo_juiced_odds"]]

,batting_name,prediction_label,prediction_score,player_team_name,theo_odds,theo_juiced_odds
288,Andrew Benintendi,1,0.7800,Chicago White Sox,-355,-370
284,Andrew Vaughn,1,0.7756,Chicago White Sox,-346,-361
141,Nathaniel Lowe,1,0.7753,Texas Rangers,-345,-360
293,Luis Robert Jr.,1,0.7592,Chicago White Sox,-315,-330
96,Austin Riley,1,0.7531,Atlanta Braves,-305,-320
...,...,...,...,...,...,...
38,Ty France,1,0.6559,Seattle Mariners,-191,-206
357,Bryson Stott,1,0.6534,Philadelphia Phillies,-189,-204
41,Julio Rodriguez,1,0.6519,Seattle Mariners,-187,-202
197,Hunter Renfroe,1,0.6516,Los Angeles Angels,-187,-202


In [ ]:
Daily_Hit_Prediction.to_csv("labeled-"+file_string)
files.download("labeled-"+file_string)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>